# DS 2500 HW 7

Due: Mon Apr 03 @ 11:59PM

### Submission Instructions
Please submit both of the following to the corresponding [gradescope](https://www.gradescope.com/courses/478298) assignment:
- this `.ipynb` file
     -  <span style="color:red">give a fresh Kernel > Restart & Run All just before uploading</span>
         - there is no autograder for hw7, so this step is extra important here!

- a `.py` file consistent with your `.ipynb`
    - `File > Download as ...`

### Tips for success
- Start early
- Make use of [Piazza](https://course.ccs.neu.edu/ds2500/admin_piazza.html)
- Make use of [Office Hours](https://course.ccs.neu.edu/ds2500/office_hours.html)
- Remember that [Documentation / style counts for credit](https://course.ccs.neu.edu/ds2500/python_style.html)
- [No student may view or share their ungraded homework with another](https://course.ccs.neu.edu/ds2500/syllabus.html#academic-integrity-and-conduct)

| part                                        |       | ex cred   |   part total |
|:--------------------------------------------|:------|:----------|-------------:|
| Part 1: `BayesNetwork.add_prior_node`       | 20.0  |           |           20 |
| Part 2: `BayesNetwork.add_conditional_node` | 25.0  |           |           25 |
| Part 3: `BayesNet.get_prob`                 | 20.0  |           |           20 |
| Part 4: `BayesNet.get_conditional_prob`     | 15.0  |           |           15 |
| Part 5: Gardening                           | 15.0  |           |           15 |
| Part 6: Memory Analysis                     | 5.0   |           |            5 |
| Part 7: Build-your-own                      |       | 4.0       |            4 |
| total                                       | 100.0 | 4.0       |          104 |

# Suggestions:

- only modify the code in the cell immediately below
    - modifying the tests can alter the intended behavior of the test
- test your code by giving a fresh restart & run for each run
    - the tests are built to be run in the given sequence, running a code cell twice 
    
# Hints:

- this `BayesNetwork` class operates just like the "manual spreadsheet" computation shown in class.  Before diving into the syntax and programming challenge of building it be sure you're comfortable with the mathematics and "manual" computation method shown in class first.
- [hw7_hint](hw7_hint.ipynb) has a few constructions which could be useful   


In [1]:
from copy import copy

import pandas as pd


class BayesNetwork:
    """ Bayes Net, computes full joint table

    Attributes:
        df_joint (pd.DataFrame): a column per random variable plus another col
            for probability.  each row contains the outcomes of the
            corresponding random variable or the joint prob of entire row
    """

    def __init__(self):
        # note: we specify type of prob as float with 1.0 below
        self.df_joint = pd.DataFrame({'prob': [1.0]})

    def add_prior_node(self, rv_name, prob_dist):
        """ adds a nodes to joint distribution table

        Args:
            rv_name (str): name of random variable (must be unique in df_joint)
            prob_dist (dict): keys are outcomes of random variable, values are
                probability of each
        """
        assert rv_name not in self.df_joint.columns, \
            f'non-unique node: {rv_name}'
        
        # initialize list that will compile all rows of dataframe 
        row_list = list()
        
        # go through outcomes and probabilities in dict to make each new row
        for key, value in prob_dist.items():
            new_row = {'prob': value, rv_name: key}
            row_list.append(new_row)
        
        # create dataframe using list of dictionaries for all the rows
        self.df_joint = pd.DataFrame(row_list, columns = ['prob', rv_name])
        
    def add_conditional_node(self, cond_dist):
        """ adds a nodes to joint distribution table

        Args:
            cond_dist (ConditionalProb): a conditional probability of some new
                random variable.  (conditioned on random variables already in
                df_joint)
        """
        # check that all conditioned variables are in joint already
        assert set(cond_dist.condition_list).issubset(self.df_joint.columns), \
            f'condition rvs not in joint table: {cond_dist.condition_list}'
        
        # check that target variable is not in joint already
        assert cond_dist.target not in self.df_joint.columns, \
            f'random variable already in network: {cond_dist.target}'
        
        # initialize list that will include all rows
        row_list = list()
        
        # go through each row of dataframe
        for idx, row in self.df_joint.iterrows():
            
            # the number of conditions given in list will determine the tuple of outcomes to find conditional probability
            for condition in cond_dist.condition_list:
                if len(cond_dist.condition_list) > 1:
                    cond = (row[-2:][0],row[-2:][1])
                else:
                    cond = (row[condition],)
            
            # extract probability of row
            prob = row['prob'] 
            
            # extract conditional probability using tuple of conditions from list
            cond_prob = cond_dist.cond_prob_dict[cond] 
            
            # go through each outcome and probability and create new rows with updated probabilities
            for key, value in cond_prob.items(): 
                new_prob = value * prob
                new_row = copy(row)
                new_row['prob'] = new_prob
                new_row[cond_dist.target] = key
                row_list.append(new_row)
                
        # create new dataframe based on updated rows
        self.df_joint = pd.DataFrame(row_list)
        

    def get_prob(self, state):
        """ sums all rows which satisfy state (marginalization)

        Args:
            state (dict): keys are random variable, values are corresponding
                outcomes
                
        Returns:
            prob (float): probability of the given state
        """
        # initialize probability variable
        prob = 0
        
        # go through each row in dataframe
        for idx, row in self.df_joint.iterrows():
            # convert each row into a dictionary
            row_dict = row.to_dict()
            
            # check if state dict is contained within the row dict
            if set(state.items()).issubset(row_dict.items()):
                # if yes, add probability of that row
                prob += row['prob']
        return prob

    def get_conditional_prob(self, state, condition):
        """ computes conditional probability of state given condition:

        P(ABC|XYZ) = P(ABCXYZ) / P(XYZ)

        above ABC are state variables while XYZ are conditional variables

        Args:
            state (dict): keys are random variable, values are corresponding
                outcomes
            condition (dict): keys are random variable, values are
                corresponding outcomes
                
        Returns:
            prob (float): probability of the given state given condition
        """
        # check that no variable is in state & conditional
        rv_double = set(state.keys()).intersection(condition.keys())
        assert not rv_double, \
            f'same random variable before & after conditional: {rv_double}'
        
        # compute P(ABCXYZ) and P(XYZ)
        num = self.get_prob(state|condition)
        den = self.get_prob(condition)
        
        # compute ratio to find P(ABC|XYZ)
        prob = num/den
        return prob

# Part 1: `BayesNetwork.add_prior_node` (20 points)

We validate whether the nodes have been added properly by constructing a known example: 

<img src="https://miro.medium.com/max/640/1*9OsQV0PqM2juaOtGqoRISw.jpeg" width=500>

and comparing output `bayes_net.df_joint` to expected dataframes, which are stored in the [expected_csv](expected_csv) folder.

In [2]:
# for example, after adding the cloudy node to the network, bayes_net.df_joint should look as below:
df_expected = pd.read_csv('expected_csv/prob_cloudy.csv', index_col=False)
df_expected

,prob,Cloudy
0,0.5,c0
1,0.5,c1


In [3]:
# build bayes net with cloudy node
bayes_net = BayesNetwork()
bayes_net.add_prior_node('Cloudy', prob_dist={'c0': .5, 'c1': .5})

# manually check output dataframe (just this first time, to see how to debug below)
bayes_net.df_joint

,prob,Cloudy
0,0.5,c0
1,0.5,c1


In [4]:
from df_compare import assert_df_equal_no_idx

# automatically compare expected to actual dataframe
# (it ends up being somewhat challenging to do given that we 
# can shuffle order of cols or rows while the two are still
# equivilent, for our purposes ... see df_compare.py for details,
# but it isn't necessary to complete the assignment)
assert_df_equal_no_idx(bayes_net.df_joint, df_expected)

# Part 2: `BayesNetwork.add_conditional_node` (25 points)

Hint:
- Inspect and study the given output DataFrames via their [expected_csv](expected_csv) before implementing!

In [5]:
from conditional import ConditionalProb

# add rain conditional prob
cond_prob_rain = \
    ConditionalProb(target='Rain',
                    condition_list=['Cloudy'],
                    cond_prob_dict={('c1',): {'r1': .8, 'r0': .2},
                                    ('c0',): {'r1': .2, 'r0': .8}})
bayes_net.add_conditional_node(cond_prob_rain)
# check that rain conditional prob was added properly
df_joint_expected = pd.read_csv('expected_csv/prob_cloudy_rain.csv', index_col=False)
assert_df_equal_no_idx(df_joint_expected, bayes_net.df_joint)

In [6]:
df_joint_expected

,prob,Cloudy,Rain
0,0.1,c0,r1
1,0.4,c0,r0
2,0.4,c1,r1
3,0.1,c1,r0


In [7]:
# add sprinkler conditional prob
cond_prob_sprinkler = \
    ConditionalProb(target='Sprinkler',
                    condition_list=['Cloudy'],
                    cond_prob_dict={('c1',): {'s1': .1, 's0': .9},
                                    ('c0',): {'s1': .5, 's0': .5}})
bayes_net.add_conditional_node(cond_prob_sprinkler)
# check that sprinkler conditional prob was added properly
df_joint_expected = pd.read_csv('expected_csv/prob_cloudy_rain_sprinkler.csv', index_col=False)
assert_df_equal_no_idx(df_joint_expected, bayes_net.df_joint)

In [8]:
df_joint_expected

,prob,Cloudy,Rain,Sprinkler
0,0.05,c0,r1,s1
1,0.05,c0,r1,s0
2,0.20,c0,r0,s1
3,0.20,c0,r0,s0
4,0.04,c1,r1,s1
5,0.36,c1,r1,s0
6,0.01,c1,r0,s1
7,0.09,c1,r0,s0


In [9]:
# add wet grass conditional prob
cond_prob_grass_wet = \
    ConditionalProb(target='WetGrass',
                    condition_list=['Rain', 'Sprinkler'],
                    cond_prob_dict={('r1', 's1'): {'w1': .99, 'w0': .01},
                                    ('r0', 's1'): {'w1': 0.9, 'w0': .1},
                                    ('r1', 's0'): {'w1': 0.9, 'w0': .1},
                                    ('r0', 's0'): {'w1': 0.0, 'w0': 1}})
bayes_net.add_conditional_node(cond_prob_grass_wet)

# check that wet grass conditional prob was added properly
df_joint_expected = pd.read_csv('expected_csv/prob_cloudy_rain_sprinkler_grass.csv', index_col=False)
assert_df_equal_no_idx(df_joint_expected, bayes_net.df_joint)

# Part 3: `BayesNet.get_prob` (20 points)

In [10]:
from math import isclose

assert isclose(bayes_net.get_prob({'Cloudy': 'c1'}), .5)

assert isclose(bayes_net.get_prob({'Sprinkler': 's1', 'Cloudy': 'c1'}), .05)
assert isclose(bayes_net.get_prob({'Sprinkler': 's1', 'Cloudy': 'c0'}), .25)
assert isclose(bayes_net.get_prob({'Sprinkler': 's1'}), .3)

assert isclose(bayes_net.get_prob({'Rain': 'r1', 'Cloudy': 'c1'}), .4)
assert isclose(bayes_net.get_prob({'Rain': 'r1', 'Cloudy': 'c0'}), .1)
assert isclose(bayes_net.get_prob({'Rain': 'r1'}), .5)

#### extra math note (not needed for HW completion, helpful for probability fluency though)

The chunks of three assert statements immediately above demonstrate marginalization: 

- there's only two ways sprinkler is on: 
    - when its cloudy or clear outside (.3 = .05 + .25)
- there's only two ways its raining:     
    - when its cloudy or clear outside (.5 = .1 + .4)

# Part 4: `BayesNet.get_conditional_prob` (15 points)

To validate `.get_conditional_prob()` we reproduce known conditional probs from the bayes net definition:

In [11]:
# whats the prob the sprinkler is on given its cloudy?
assert isclose(bayes_net.get_conditional_prob(state={'Sprinkler': 's1'}, condition={'Cloudy': 'c1'}), .1)

In [12]:
# whats the prob its not raining given its not cloudy?
assert isclose(bayes_net.get_conditional_prob(state={'Rain': 'r0'}, condition={'Cloudy': 'c0'}), .8)

In [13]:
# whats the prob lawn is wet given sprinkler is on and its raining?
assert isclose(bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Sprinkler': 's1',
                                                                                    'Rain': 'r1'}), .99)

# Part 5: Gardening (15 points)

A gardener wants their newly planted lawn to have (at least) a 70% chance of being wet while using their sprinkler as little as possible, to conserve water.  Each morning they step outside their house and observe only if it is cloudy or not.  With only this evidence, they want to know whether they must turn their sprinkler on.

- on clear days, should the gardener turn on their sprinkler?
- on cloudy days, should the gardener turn on their sprinkler?
- is it possible for the gardener to always ensure at least 70% chance of having a wet lawn?

Call a few methods of the bayes net above to investigate the questions immediately above.  Write a summary of results in 2-3 sentences which is easily understood by a garener who knows little of probability or Bayes Nets.

In [14]:
# clear days chance of rain
prob1 = bayes_net.get_conditional_prob(state={'Rain': 'r1'}, condition={'Cloudy': 'c0'})
print("Probability of rain on a clear day: ", prob1)

# chance of wet grass given no rain and clear day
prob2 = bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Cloudy': 'c0', 'Rain': 'r0'})
print("Probability of wet grass given no rain and a clear day: ", prob2)

# probability of wet grass with clear day and no sprinkler
prob3 = bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Cloudy': 'c0', 'Sprinkler': 's0'})
print("Probability of wet grass with a clear day and no sprinkler: ", prob3)


Probability of rain on a clear day:  0.2
Probability of wet grass given no rain and a clear day:  0.45
Probability of wet grass with a clear day and no sprinkler:  0.18000000000000002


In [15]:
# cloudy days chance of rain
prob4 = bayes_net.get_conditional_prob(state={'Rain': 'r1'}, condition={'Cloudy': 'c1'})
print("Probability of rain on a cloudy day: ", prob4)

# chance of wet grass given cloudy day
prob5 = bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Cloudy': 'c1'})
print("Probability of wet grass on a cloudy day: ", prob5)

# probability of wet grass with cloudy day and sprinkler off
prob6 = bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Cloudy': 'c1', 'Sprinkler': 's0'})
print("Probability of wet grass on a cloudy day and sprinkler off: ", prob4)

Probability of rain on a cloudy day:  0.8
Probability of wet grass on a cloudy day:  0.7452000000000001
Probability of wet grass on a cloudy day and sprinkler off:  0.8


In [16]:
# if sprinkler is always on
bayes_net.get_conditional_prob(state={'WetGrass': 'w1'}, condition={'Sprinkler': 's1'})

0.9269999999999999

As shown by the results above, because there is little probability of rain on a clear day and of wet grass on a clear day with no sprinkler, the gardner should turn on the sprinkler. On cloudy days, there is a high probability of rain and a 80% chance of wet grass on a cloudy day with no sprinkler, so the gardner should turn off the sprinkler. The gardner can ensure at least 70% chance of having a wet lawn by always keeping the sprinkler on. 

# Part 6: Memory Analysis (5 points)

Let's consider the liver disease bayes net example shown in class.  Assuming it has 40 total nodes, and each is binary, how much memory would it cost to store the probability column of `df_joint` as shown above?  Assume that every combination of variables must be stored as a float which uses `np.ones(1).nbytes / 1e6` megabytes of space.

Summarize your computation in 2 sentences so a non-technical reader can understand the drawback.  (Note: this memory problem lies with our implementation, there are methods to avoid it)

Hint:
- its a big number, don't try this line of code as you'll run out of memory before you get an answer:
    - `np.ones(2 ** 40).nbytes / 1e6` megabytes of space

In [17]:
import numpy as np
np.ones(1).nbytes / 1e6

8e-06

In [18]:
(np.ones(1).nbytes * 2**40) / 1e6

8796093.022208

In our cloudy/wet grass example above, there are 4 nodes and consequently 2^4 = 16 rows in the dataframe. If we have 40 total nodes, we will have 2^40 rows in the dataframe, and assuming each float uses np.ones(1).nbytes / 1e6 megabytes of space, multiplying it by 2**40 will produce 8796093.022208 megabytes of space. 

# Part 7: Build-your-own (4 ex cred pts)

Build your own Bayes Net problem!

1. Provide a graphical representation which contains a graph and all necessary distributions
    - see the thief, alarm, dog, doorbell, earthquake example in class
    - include it as an embedded image directly below
1. Implement it as a `BayesNet`
1. Write a few questions which tell a "data story".  Answer them by querying your network and interpretting results.
    - again, see the thief example in class for a "data story"
    
Grab your project team's data if you'd like :)

I'd love a few more beautiful examples for use in future coursework.  Make a super-clean figure and a compelling datastory to earn the full four points of credit.  

If you're willing to share this in future coursework (for any course or instructor) please shoot me a copy via email saying "You, or other instructors, are welcome to use this in any future course".  Also, let us know if you'd like us to cite you or whether you'd like us to give credit to an anonymous DS2500 student.  Your consent to use / share won't impact whether you score extra credit points.

<img src="https://global.discourse-cdn.com/standard10/uploads/pymc3/original/2X/7/758e4a294906db14642a2f5ee8de189e4bf1d9e0.png" width=500>

In [19]:
import pandas as pd

df_hospital = pd.read_csv('expected_csv/extracredit.csv', index_col=False)
df_hospital

,prob,Smoker,COVID,Hospital
0,0.90,s1,c1,h1
1,0.10,s1,c0,h1
2,0.90,s0,c1,h1
3,0.01,s0,c0,h1
4,0.10,s1,c1,h0
5,0.90,s1,c0,h0
6,0.10,s0,c1,h0
7,0.99,s0,c0,h0


In [20]:
bayes_net = BayesNetwork()
bayes_net.add_prior_node('Smoker', prob_dist={'s0': .25, 's1': .75})

bayes_net.df_joint

,prob,Smoker
0,0.25,s0
1,0.75,s1


In [21]:
cond_prob_covid = \
    ConditionalProb(target='COVID',
                    condition_list=['Smoker'],
                    cond_prob_dict={('s1',): {'c1': 0.1, 'c0': .9},
                                    ('s0',): {'c1': .1, 'c0': .9}})
bayes_net.add_conditional_node(cond_prob_covid)
bayes_net.df_joint

,prob,Smoker,COVID
0,0.025,s0,c1
0,0.225,s0,c0
1,0.075,s1,c1
1,0.675,s1,c0


In [22]:
cond_prob_hospital = \
    ConditionalProb(target='Hospital',
                    condition_list=['Smoker', 'COVID'],
                    cond_prob_dict={('s1', 'c1'): {'h1': .9, 'h0': .1},
                                    ('s1', 'c0'): {'h1': .1, 'h0': .9},
                                   ('s0', 'c1'): {'h1': .9, 'h0': .1},
                                   ('s0', 'c0'): {'h1': .01, 'h0': .99}})
bayes_net.add_conditional_node(cond_prob_hospital)
bayes_net.df_joint

,prob,Smoker,COVID,Hospital
0,0.02250,s0,c1,h1
0,0.00250,s0,c1,h0
0,0.00225,s0,c0,h1
0,0.22275,s0,c0,h0
1,0.06750,s1,c1,h1
1,0.00750,s1,c1,h0
1,0.06750,s1,c0,h1
1,0.60750,s1,c0,h0


- Do a higher percentage of smokers or COVID patients take up hospital space?
- Do smokers have a higher chance of getting COVID than non-smokers?
- Are more smokers likely to be in the hospital without having COVID?

In [23]:
prob1 = bayes_net.get_conditional_prob(state={'Hospital': 'h1'}, condition={'Smoker': 's1'})
print("Probability of being in the hospital as a smoker:", prob1)

prob2 = bayes_net.get_conditional_prob(state={'Hospital': 'h1'}, condition={'COVID': 'c1'})
print("Probability of being in the hospital as COVID patient:", prob2)

Probability of being in the hospital as a smoker: 0.18000000000000002
Probability of being in the hospital as COVID patient: 0.8999999999999999


In [24]:
prob3 = bayes_net.get_conditional_prob(state={'COVID': 'c1'}, condition={'Smoker': 's1'})
print("Probability of being a smoker and having COVID:", prob3)

prob4 = bayes_net.get_conditional_prob(state={'COVID': 'c1'}, condition={'Smoker': 's0'})
print("Probability of being a non-smoker and having COVID:", prob4)

Probability of being a smoker and having COVID: 0.10000000000000003
Probability of being a non-smoker and having COVID: 0.1


In [25]:
prob5 = bayes_net.get_conditional_prob(state={'Hospital': 'h1'}, condition={'Smoker': 's1','COVID':'c0'})
print("Probability of being a smoker and being in the hospital without COVID:", prob5)

Probability of being a smoker and being in the hospital without COVID: 0.1


Based on the results above, a higher percentage of COVID patients take up hospital space. Furthermore, based on the data, the probability of having COVID as a smoker is the same as having it as a non-smoker. Finally, the probability of a smoker being in the hospital without having COVID is only 0.1, so it is not very likely.